In [1]:
import nltk
import numpy as np
import pandas as pd
import random
import time
import gzip
import ast
import operator
import sys
import pickle as pkl
from collections import Counter
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
import keras
#from keras.models import Model
#from keras.models import load_model
from keras.layers import *
from nltk.chunk import conlltags2tree, tree2conlltags
from nltk import word_tokenize, pos_tag, ne_chunk

Using TensorFlow backend.


# Predict

In [2]:
tf.__version__

'2.1.0'

In [3]:
keras.__version__

'2.3.1'

In [4]:
np.random.seed(1337)  # for reproducibility
DATA_PATH = "bi-lstm-data/"
EMBEDDINGS_PATH = DATA_PATH+'embeddings.pkl.gz'
TRAIN_SET_PATH = DATA_PATH+'train_set.pkl.gz'
TEST_SET_PATH = DATA_PATH+'test_set.csv'
MODEL_PATH = DATA_PATH+'f1e5-bi-lstm-model.h5'
PREDICTIONS_PATH = DATA_PATH+'best-match.csv'

In [5]:
# Load a compiles Bi-LSTM model
model = keras.models.load_model(MODEL_PATH)

In [6]:
# Load embeddings
f = gzip.open(EMBEDDINGS_PATH , 'rb')
embeddings = pkl.load(f)
f.close()

# Load label encoddings
label2Idx = embeddings['label2Idx']

#Inverse label mapping
idx2Label = {v: k for k, v in label2Idx.items()}
print(label2Idx,'\n---\n', idx2Label)

{'I-MISC': 0, 'B-LOC': 1, 'B-ORG': 2, 'O': 3, 'I-ORG': 4, 'I-LOC': 5, 'I-PER': 6, 'B-PER': 7, 'B-MISC': 8} 
---
 {0: 'I-MISC', 1: 'B-LOC', 2: 'B-ORG', 3: 'O', 4: 'I-ORG', 5: 'I-LOC', 6: 'I-PER', 7: 'B-PER', 8: 'B-MISC'}


In [7]:
# Load test set
df = pd.read_csv(TEST_SET_PATH, sep=';', index_col=[0])
df.head(3)

,Titulo,Sintesis,Texto,titulo_tokens,sintesis_tokens,texto_tokens,titulo_word_inds,titulo_case_inds,sintesis_word_inds,sintesis_case_inds,texto_word_inds,texto_case_inds
0,Exigen a TransCanada transparentar proyecto de...,Presidentes de las sociedades cooperativas pes...,Presidentes de las sociedades cooperativas pes...,"[['Exigen', 'a', 'TransCanada', 'transparentar...","[['Presidentes', 'de', 'las', 'sociedades', 'c...","[['Presidentes', 'de', 'las', 'sociedades', 'c...","[[23151, 8, 56126, 25835, 128, 2, 20125]]","[[3, 1, 3, 1, 1, 1, 1]]","[[8276, 2, 11, 2597, 8975, 13703, 2, 47087, 6,...","[[3, 1, 1, 1, 1, 1, 1, 3, 1, 3, 4, 1, 1, 1, 3,...","[[8276, 2, 11, 2597, 8975, 13703, 2, 47087, 6,...","[[3, 1, 1, 1, 1, 1, 1, 3, 1, 3, 4, 1, 1, 1, 3,..."
1,Violento fin de semana en dos estados de Mexic...,Los casos ocurrieron en Guanajuato y Chihuahua...,Los casos ocurrieron en Guanajuato y Chihuahua...,"[['Violento', 'fin', 'de', 'semana', 'en', 'do...","[['Los', 'casos', 'ocurrieron', 'en', 'Guanaju...","[['Los', 'casos', 'ocurrieron', 'en', 'Guanaju...","[[42177, 127, 2, 435, 4, 38, 1425, 2, 16336, 1...","[[3, 1, 1, 1, 1, 1, 1, 1, 3, 4, 0, 1]]","[[35, 301, 8038, 4, 5840, 6, 6319, 1], [5285, ...","[[3, 1, 1, 1, 3, 1, 3, 4], [3, 1, 5, 1, 1, 1, ...","[[35, 301, 8038, 4, 5840, 6, 6319, 1], [5285, ...","[[3, 1, 1, 1, 3, 1, 3, 4], [3, 1, 5, 1, 1, 1, ..."
2,Expulsan a 3 personas en Pahuatlan por trabaja...,Los empleados de la TransCanada regresaban de ...,Los empleados de la TransCanada regresaban de ...,"[['Expulsan', 'a', '3', 'personas', 'en', 'Pah...","[['Los', 'empleados', 'de', 'la', 'TransCanada...","[['Los', 'empleados', 'de', 'la', 'TransCanada...","[[48750, 8, 1, 71, 4, 66437, 13, 926, 4, 20125]]","[[3, 1, 0, 1, 1, 3, 1, 1, 1, 1]]","[[35, 1659, 2, 3, 56126, 21451, 2, 855, 103, 2...","[[3, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[35, 1659, 2, 3, 56126, 21451, 2, 855, 103, 2...","[[3, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."


In [8]:
def predict(model, word_indexes, case_indexes):
    """Predict word labels"""
    
    predictions = []
    
    #print('word_indexes ', word_indexes)
    #print('case_indexes ',case_indexes)
    word_inds_list = ast.literal_eval(word_indexes)
    case_inds_list = ast.literal_eval(case_indexes)
    #print('word_inds_list ', word_inds_list)
    #print('case_inds_list ', case_inds_list)

    
    for idx, word_inds in enumerate(word_inds_list):
        word_inds = np.asarray([word_inds])
        case_inds = np.asarray([case_inds_list[idx]])
        #print(type(word_inds), word_inds)
        #print(type(case_inds), case_inds)
        
        pred = model.predict([word_inds, case_inds], verbose=False)[0]
        #print(pred)
        pred = pred.argmax(axis=-1) #Predict the classes  
        #print(pred)
        predictions.append(pred.tolist())
    return(predictions)
        

In [9]:
label2Idx['B-LOC'], label2Idx['I-LOC'] 

(1, 5)

In [10]:
def label_tokens(tokens_list, labels_list, idx2Label):
    """Add to  words their IOB labels"""
    result = []
    tokens_list = ast.literal_eval(tokens_list)
    for idx, tokens in enumerate(tokens_list):
        sent = []
        labels = labels_list[idx]
        for idx, token in enumerate(tokens):
            sent.append((token, idx2Label[labels[idx]]))
        result.append(sent)
    return (result)

In [11]:
def find_predicted_locations(tokens_list, labels_list, label2Idx):
    """For given label encodings find words with location tags"""
    result = []
    tokens_list = ast.literal_eval(tokens_list)
    for idx, tokens in enumerate(tokens_list):
        labels = labels_list[idx]
        for idx, token in enumerate(tokens):
            if(labels[idx] == label2Idx['B-LOC'] or labels[idx] == label2Idx['I-LOC']):
                result.append((token, idx2Label[labels[idx]]))
    return (result)
        

In [12]:
# test
indx = 3
column = 'Sintesis'
prefix = column.lower()
word_inds_name = prefix + '_word_inds'
case_inds_name = prefix + '_case_inds'
tokens_name = prefix + '_tokens'
sent = df[column][indx] 
tokens = df[tokens_name][indx]
print('*** sent: ',sent)
print('*** tokens: ', tokens)
word_inds = df[word_inds_name][indx]
case_inds = df[case_inds_name][indx]
print('*** word index: ',word_inds)
print('*** case index: ',case_inds)
label_list = predict(model, word_inds, case_inds)
print('*** labels: ',label_list)
labeled = label_tokens(tokens, label_list, idx2Label)
print('*** labeled: ', labeled)
locations = find_predicted_locations(tokens, label_list, label2Idx)
print('*** locations: ', locations)

*** sent:  Los hechos se registraron la madrugada de este lunes en el municipio de Jimenez
*** tokens:  [['Los', 'hechos', 'se', 'registraron', 'la', 'madrugada', 'de', 'este', 'lunes', 'en', 'el', 'municipio', 'de', 'Jimenez']]
*** word index:  [[35, 834, 12, 4628, 3, 3223, 2, 34, 1199, 4, 5, 340, 2, 31933]]
*** case index:  [[3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3]]
*** labels:  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]]
*** labeled:  [[('Los', 'I-MISC'), ('hechos', 'I-MISC'), ('se', 'I-MISC'), ('registraron', 'I-MISC'), ('la', 'I-MISC'), ('madrugada', 'I-MISC'), ('de', 'I-MISC'), ('este', 'I-MISC'), ('lunes', 'I-MISC'), ('en', 'I-MISC'), ('el', 'I-MISC'), ('municipio', 'I-MISC'), ('de', 'I-MISC'), ('Jimenez', 'B-ORG')]]
*** locations:  []


# Predict IOB tags

In [13]:
# Predict class labels
df['titulo_labels'] = df.apply(lambda x: predict(model, x['titulo_word_inds'],
                                                 x['titulo_case_inds']), axis=1) 

In [14]:
%%time
df['sintesis_labels'] = df.apply(lambda x: predict(model, x['sintesis_word_inds'], 
                                                   x['sintesis_case_inds']), axis=1)     

CPU times: user 47.3 s, sys: 10.3 s, total: 57.6 s
Wall time: 27.6 s


In [15]:
%%time
df['texto_labels'] = df.apply(lambda x: predict(model, x['texto_word_inds'], 
                                                x['texto_case_inds']), axis=1)  
df.head()

CPU times: user 4min 57s, sys: 1min 5s, total: 6min 3s
Wall time: 2min 52s


,Titulo,Sintesis,Texto,titulo_tokens,sintesis_tokens,texto_tokens,titulo_word_inds,titulo_case_inds,sintesis_word_inds,sintesis_case_inds,texto_word_inds,texto_case_inds,titulo_labels,sintesis_labels,texto_labels
0,Exigen a TransCanada transparentar proyecto de...,Presidentes de las sociedades cooperativas pes...,Presidentes de las sociedades cooperativas pes...,"[['Exigen', 'a', 'TransCanada', 'transparentar...","[['Presidentes', 'de', 'las', 'sociedades', 'c...","[['Presidentes', 'de', 'las', 'sociedades', 'c...","[[23151, 8, 56126, 25835, 128, 2, 20125]]","[[3, 1, 3, 1, 1, 1, 1]]","[[8276, 2, 11, 2597, 8975, 13703, 2, 47087, 6,...","[[3, 1, 1, 1, 1, 1, 1, 3, 1, 3, 4, 1, 1, 1, 3,...","[[8276, 2, 11, 2597, 8975, 13703, 2, 47087, 6,...","[[3, 1, 1, 1, 1, 1, 1, 3, 1, 3, 4, 1, 1, 1, 3,...","[[0, 0, 7, 0, 0, 0, 0]]","[[0, 0, 0, 0, 0, 0, 0, 2, 0, 7, 0, 0, 0, 0, 7,...","[[0, 0, 0, 0, 0, 0, 0, 2, 0, 7, 0, 0, 0, 0, 7,..."
1,Violento fin de semana en dos estados de Mexic...,Los casos ocurrieron en Guanajuato y Chihuahua...,Los casos ocurrieron en Guanajuato y Chihuahua...,"[['Violento', 'fin', 'de', 'semana', 'en', 'do...","[['Los', 'casos', 'ocurrieron', 'en', 'Guanaju...","[['Los', 'casos', 'ocurrieron', 'en', 'Guanaju...","[[42177, 127, 2, 435, 4, 38, 1425, 2, 16336, 1...","[[3, 1, 1, 1, 1, 1, 1, 1, 3, 4, 0, 1]]","[[35, 301, 8038, 4, 5840, 6, 6319, 1], [5285, ...","[[3, 1, 1, 1, 3, 1, 3, 4], [3, 1, 5, 1, 1, 1, ...","[[35, 301, 8038, 4, 5840, 6, 6319, 1], [5285, ...","[[3, 1, 1, 1, 3, 1, 3, 4], [3, 1, 5, 1, 1, 1, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0]]","[[0, 0, 0, 0, 2, 0, 2, 0], [1, 0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 2, 0, 2, 0], [1, 0, 0, 0, 0, 0, ..."
2,Expulsan a 3 personas en Pahuatlan por trabaja...,Los empleados de la TransCanada regresaban de ...,Los empleados de la TransCanada regresaban de ...,"[['Expulsan', 'a', '3', 'personas', 'en', 'Pah...","[['Los', 'empleados', 'de', 'la', 'TransCanada...","[['Los', 'empleados', 'de', 'la', 'TransCanada...","[[48750, 8, 1, 71, 4, 66437, 13, 926, 4, 20125]]","[[3, 1, 0, 1, 1, 3, 1, 1, 1, 1]]","[[35, 1659, 2, 3, 56126, 21451, 2, 855, 103, 2...","[[3, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[35, 1659, 2, 3, 56126, 21451, 2, 855, 103, 2...","[[3, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 2, 0, 0, 0, 0]]","[[0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,Enfrentamiento entre civiles y militares deja ...,Los hechos se registraron la madrugada de este...,se reporta una persona detenida.,"[['Enfrentamiento', 'entre', 'civiles', 'y', '...","[['Los', 'hechos', 'se', 'registraron', 'la', ...","[['se', 'reporta', 'una', 'persona', 'detenida...","[[32119, 33, 1148, 6, 872, 1877, 1, 1752, 4, 6...","[[3, 1, 1, 1, 1, 1, 0, 1, 1, 3]]","[[35, 834, 12, 4628, 3, 3223, 2, 34, 1199, 4, ...","[[3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3]]","[[12, 12611, 17, 429, 8479, 1]]","[[1, 1, 1, 1, 1, 4]]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 2]]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]]","[[0, 0, 0, 0, 0, 0]]"
4,Identifican a joven localizada sin vida en Gua...,La joven madre fue identificada como Rosario J...,La joven madre fue identificada como Rosario J...,"[['Identifican', 'a', 'joven', 'localizada', '...","[['La', 'joven', 'madre', 'fue', 'identificada...","[['La', 'joven', 'madre', 'fue', 'identificada...","[[29293, 8, 779, 7394, 54, 133, 4, 57330]]","[[3, 1, 1, 1, 1, 1, 1, 3]]","[[24, 779, 842, 32, 9692, 23, 3764, 17412, 1, ...","[[3, 1, 1, 1, 1, 1, 3, 3, 2, 2, 4, 1, 1, 3, 4,...","[[24, 779, 842, 32, 9692, 23, 3764, 17412, 1, ...","[[3, 1, 1, 1, 1, 1, 3, 3, 2, 2, 4, 1, 1, 3, 4,...","[[0, 0, 0, 0, 0, 0, 0, 2]]","[[0, 0, 0, 0, 0, 0, 1, 4, 4, 4, 0, 0, 0, 2, 0,...","[[0, 0, 0, 0, 0, 0, 1, 4, 4, 4, 0, 0, 0, 2, 0,..."


# Find locations

In [16]:
# For every text field convert IOB nymeric labels to IOB text tags
df['titulo_tags'] = df.apply(lambda x: label_tokens(x['titulo_tokens'], 
                                                    x['titulo_labels'], idx2Label), axis = 1)

In [17]:
df['sintesis_tags'] = df.apply(lambda x: label_tokens(x['sintesis_tokens'], 
                                                      x['sintesis_labels'], idx2Label), axis = 1)

In [18]:
%%time
df['texto_tags'] = df.apply(lambda x: label_tokens(x['texto_tokens'], 
                                                   x['texto_labels'], idx2Label), axis = 1)

CPU times: user 1.18 s, sys: 23.9 ms, total: 1.21 s
Wall time: 1.21 s


In [19]:
df.iloc[0]

Titulo                Exigen a TransCanada transparentar proyecto de...
Sintesis              Presidentes de las sociedades cooperativas pes...
Texto                 Presidentes de las sociedades cooperativas pes...
titulo_tokens         [['Exigen', 'a', 'TransCanada', 'transparentar...
sintesis_tokens       [['Presidentes', 'de', 'las', 'sociedades', 'c...
texto_tokens          [['Presidentes', 'de', 'las', 'sociedades', 'c...
titulo_word_inds              [[23151, 8, 56126, 25835, 128, 2, 20125]]
titulo_case_inds                                [[3, 1, 3, 1, 1, 1, 1]]
sintesis_word_inds    [[8276, 2, 11, 2597, 8975, 13703, 2, 47087, 6,...
sintesis_case_inds    [[3, 1, 1, 1, 1, 1, 1, 3, 1, 3, 4, 1, 1, 1, 3,...
texto_word_inds       [[8276, 2, 11, 2597, 8975, 13703, 2, 47087, 6,...
texto_case_inds       [[3, 1, 1, 1, 1, 1, 1, 3, 1, 3, 4, 1, 1, 1, 3,...
titulo_labels                                   [[0, 0, 7, 0, 0, 0, 0]]
sintesis_labels       [[0, 0, 0, 0, 0, 0, 0, 2, 0, 7, 0, 0, 0, 0

In [20]:
# test
indx = 17
column = 'Sintesis'
prefix = column.lower()
tokens_name = prefix + '_tokens'
labels_name = prefix + '_labels'
print(df[column][indx])
print(find_predicted_locations(df[tokens_name][indx], df[labels_name][indx], label2Idx))

Celaya, Gto.- Un hombre fue asesinado de varios disparos de arma de fuego en la colonia Gobernadores, perteneciente al municipio de Celaya.
[('Celaya', 'B-LOC'), ('Gto.-', 'B-LOC')]


In [21]:
# For every text field filter out labels associated with locations: B-LOC, I-LOC
df['titulo_locations'] = df.apply(lambda x: find_predicted_locations(x['titulo_tokens'], x['titulo_labels'], label2Idx), axis = 1)

In [22]:
df['sintesis_locations'] = df.apply(lambda x: find_predicted_locations(x['sintesis_tokens'], x['sintesis_labels'], label2Idx), axis = 1)

In [23]:
df['texto_locations'] = df.apply(lambda x: find_predicted_locations(x['texto_tokens'], x['texto_labels'], label2Idx), axis = 1)

In [24]:
df.head()

,Titulo,Sintesis,Texto,titulo_tokens,sintesis_tokens,texto_tokens,titulo_word_inds,titulo_case_inds,sintesis_word_inds,sintesis_case_inds,...,texto_case_inds,titulo_labels,sintesis_labels,texto_labels,titulo_tags,sintesis_tags,texto_tags,titulo_locations,sintesis_locations,texto_locations
0,Exigen a TransCanada transparentar proyecto de...,Presidentes de las sociedades cooperativas pes...,Presidentes de las sociedades cooperativas pes...,"[['Exigen', 'a', 'TransCanada', 'transparentar...","[['Presidentes', 'de', 'las', 'sociedades', 'c...","[['Presidentes', 'de', 'las', 'sociedades', 'c...","[[23151, 8, 56126, 25835, 128, 2, 20125]]","[[3, 1, 3, 1, 1, 1, 1]]","[[8276, 2, 11, 2597, 8975, 13703, 2, 47087, 6,...","[[3, 1, 1, 1, 1, 1, 1, 3, 1, 3, 4, 1, 1, 1, 3,...",...,"[[3, 1, 1, 1, 1, 1, 1, 3, 1, 3, 4, 1, 1, 1, 3,...","[[0, 0, 7, 0, 0, 0, 0]]","[[0, 0, 0, 0, 0, 0, 0, 2, 0, 7, 0, 0, 0, 0, 7,...","[[0, 0, 0, 0, 0, 0, 0, 2, 0, 7, 0, 0, 0, 0, 7,...","[[(Exigen, I-MISC), (a, I-MISC), (TransCanada,...","[[(Presidentes, I-MISC), (de, I-MISC), (las, I...","[[(Presidentes, I-MISC), (de, I-MISC), (las, I...",[],[],"[(Gasoducto, I-LOC), (Citlali, B-LOC), (Refiri..."
1,Violento fin de semana en dos estados de Mexic...,Los casos ocurrieron en Guanajuato y Chihuahua...,Los casos ocurrieron en Guanajuato y Chihuahua...,"[['Violento', 'fin', 'de', 'semana', 'en', 'do...","[['Los', 'casos', 'ocurrieron', 'en', 'Guanaju...","[['Los', 'casos', 'ocurrieron', 'en', 'Guanaju...","[[42177, 127, 2, 435, 4, 38, 1425, 2, 16336, 1...","[[3, 1, 1, 1, 1, 1, 1, 1, 3, 4, 0, 1]]","[[35, 301, 8038, 4, 5840, 6, 6319, 1], [5285, ...","[[3, 1, 1, 1, 3, 1, 3, 4], [3, 1, 5, 1, 1, 1, ...",...,"[[3, 1, 1, 1, 3, 1, 3, 4], [3, 1, 5, 1, 1, 1, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0]]","[[0, 0, 0, 0, 2, 0, 2, 0], [1, 0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 2, 0, 2, 0], [1, 0, 0, 0, 0, 0, ...","[[(Violento, I-MISC), (fin, I-MISC), (de, I-MI...","[[(Los, I-MISC), (casos, I-MISC), (ocurrieron,...","[[(Los, I-MISC), (casos, I-MISC), (ocurrieron,...",[],"[(Mas, B-LOC)]","[(Mas, B-LOC), (Rocio, B-LOC), (Mayo, B-LOC), ..."
2,Expulsan a 3 personas en Pahuatlan por trabaja...,Los empleados de la TransCanada regresaban de ...,Los empleados de la TransCanada regresaban de ...,"[['Expulsan', 'a', '3', 'personas', 'en', 'Pah...","[['Los', 'empleados', 'de', 'la', 'TransCanada...","[['Los', 'empleados', 'de', 'la', 'TransCanada...","[[48750, 8, 1, 71, 4, 66437, 13, 926, 4, 20125]]","[[3, 1, 0, 1, 1, 3, 1, 1, 1, 1]]","[[35, 1659, 2, 3, 56126, 21451, 2, 855, 103, 2...","[[3, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",...,"[[3, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 2, 0, 0, 0, 0]]","[[0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[(Expulsan, I-MISC), (a, I-MISC), (3, I-MISC)...","[[(Los, I-MISC), (empleados, I-MISC), (de, I-M...","[[(Los, I-MISC), (empleados, I-MISC), (de, I-M...",[],[],"[(Presidente, B-LOC), (Rogelio, B-LOC), (Turis..."
3,Enfrentamiento entre civiles y militares deja ...,Los hechos se registraron la madrugada de este...,se reporta una persona detenida.,"[['Enfrentamiento', 'entre', 'civiles', 'y', '...","[['Los', 'hechos', 'se', 'registraron', 'la', ...","[['se', 'reporta', 'una', 'persona', 'detenida...","[[32119, 33, 1148, 6, 872, 1877, 1, 1752, 4, 6...","[[3, 1, 1, 1, 1, 1, 0, 1, 1, 3]]","[[35, 834, 12, 4628, 3, 3223, 2, 34, 1199, 4, ...","[[3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3]]",...,"[[1, 1, 1, 1, 1, 4]]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 2]]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]]","[[0, 0, 0, 0, 0, 0]]","[[(Enfrentamiento, I-MISC), (entre, I-MISC), (...","[[(Los, I-MISC), (hechos, I-MISC), (se, I-MISC...","[[(se, I-MISC), (reporta, I-MISC), (una, I-MIS...",[],[],[]
4,Identifican a joven localizada sin vida en Gua...,La joven madre fue identificada como Rosario J...,La joven madre fue identificada como Rosario J...,"[['Identifican', 'a', 'joven', 'localizada', '...","[['La', 

In [25]:
len(df)

2251

In [26]:
# Select not empty 'titulo' locations
df_titulo_loc = df[ df['titulo_locations'].str.len() > 0 ]
print('*** Len: ',len(df_titulo_loc))
#df_titulo_loc.head()

*** Len:  286


In [27]:
# Select not empty 'sintesis' locations
df_sintesis_loc = df[ df['sintesis_locations'].str.len() > 0 ]
print('*** Len: ',len(df_sintesis_loc))
#df_sintesis_loc.head()

*** Len:  837


In [28]:
# Select not empty 'texto' locations
df_texto_loc = df[ df['texto_locations'].str.len() > 0 ]
print('*** Len: ',len(df_texto_loc))
#df_texto_loc.head()

*** Len:  1833


In [29]:
tit_pcnt = 1086.0 / 2251 * 100
sin_pcnt = 1317.0 / 2251 * 100
txt_pcnt = 2057.0 / 2251 * 100
tit_pcnt, sin_pcnt, txt_pcnt 

(48.24522434473567, 58.50733007552199, 91.38160817414483)

In [30]:
def bestMatch(loc_tuples):
    """ Unzip list with location tuples. Sort location names by occurance count. 
    Location names with max count are then at the end of the list. 
    Return location that is last in the list. 
    """
    best_match = ''
    if(len(loc_tuples) > 0):
        #print('*** Tuples: ', loc_tuples)
        locs = [list(t) for t in zip(*loc_tuples)][0] # unzip list with location tuples
        loc_cnts = Counter(locs) # count occurances of location names 
        loc_sorted = sorted(loc_cnts.items(), key=operator.itemgetter(1)) # sort dictionary by value - location count
        loc_sorted_len = len(loc_sorted)
        #print(loc_sorted) 
        best_match = loc_sorted[loc_sorted_len - 1][0]
    return (best_match)

In [31]:
# Test
indx = 176
titulo_locs = df['titulo_locations'][indx]
sintes_locs = df['sintesis_locations'][indx]
text_locs = df['texto_locations'][indx]
#print('*** Titulo: ', df['Titulo'][indx])
print('*** titulo_locations: ', titulo_locs) 
#print('*** Sintesis: ', df['Sintesis'][indx])
print('*** sintesis_locations: ', sintes_locs)
#print('*** Texto: ', df['Texto'][indx])
print('*** texto_locations: ', text_locs) 
loc_tuples = titulo_locs + sintes_locs + text_locs
print('*** Best Match: ', bestMatch(loc_tuples))

*** titulo_locations:  []
*** sintesis_locations:  [('Villagran', 'B-LOC'), ('Juventino', 'B-LOC')]
*** texto_locations:  [('JUVENTINO', 'B-LOC'), ('Villagran', 'B-LOC'), ('Juventino', 'B-LOC'), ('Abel', 'B-LOC'), ('Las', 'I-LOC'), ('Ibarra', 'B-LOC'), ('Emiliano', 'B-LOC'), ('Plutarco', 'B-LOC'), ('III', 'I-LOC')]
*** Best Match:  Juventino


In [32]:
df['Titulo'][indx]

'Capturan a sujeto con orden de aprehension por homicidio'

In [33]:
df['Sintesis'][indx]

'La SSPE informo que la tarde-noche de este viernes, elementos de las Fuerzas de Seguridad Publica del Estado, Policia Ministerial y Policias municipales de Salamanca, Cortazar, Celaya, Comonfort, Villagran y Juventino Rosas, desplegaron un intenso operativo en diversas comunidades y colonias de esta localidad'

In [34]:
df['Texto'][indx]

'JUVENTINO ROSAS, Gto.  En operativo intermunicipal, PGJE, SSPE y policias de 6 municipios aseguraron a un hombre armado sobre quien pesa orden de aprehension por el homicidio de 3 personas. Se le decomiso un arma de fuego calibre 9 milimetros con 17 cartuchos utiles.  La SSPE informo que la tarde-noche de este viernes, elementos de las Fuerzas de Seguridad Publica del Estado, Policia Ministerial y Policias municipales de Salamanca, Cortazar, Celaya, Comonfort, Villagran y Juventino Rosas, desplegaron un intenso operativo en diversas comunidades y colonias de esta localidad.  En las inmediaciones de la comunidad de Rincon de Centeno, fue detenido un hombre, quien fue identificado como Abel N, de 41 Anos de edad, vecino de este municipio, a quien se le aseguro un arma de fuego calibre 9 milimetros, dos cargadores y 17 cartuchos utiles.  Al consultar informacion del detenido a traves del C5i, los efectivos policiales pudieron constatar que el hombre contaba con una orden de aprehension p

In [35]:
# Calculate best match of location: 
# Select location name that occurs more often then others together in 'Titulo', 'Sintesis' and 'Texto'
df['best_match'] = df.apply(lambda x: bestMatch(x['titulo_locations'] + x['sintesis_locations'] + x['texto_locations']), axis = 1)

In [36]:
# Select not empty 'best_match' locations
df_best_match = df[ df['best_match'].str.len() > 0 ]
print('*** df_best_match length: ',len(df_best_match))
df_best_match.columns

*** df_best_match length:  1882


Index(['Titulo', 'Sintesis', 'Texto', 'titulo_tokens', 'sintesis_tokens',
       'texto_tokens', 'titulo_word_inds', 'titulo_case_inds',
       'sintesis_word_inds', 'sintesis_case_inds', 'texto_word_inds',
       'texto_case_inds', 'titulo_labels', 'sintesis_labels', 'texto_labels',
       'titulo_tags', 'sintesis_tags', 'texto_tags', 'titulo_locations',
       'sintesis_locations', 'texto_locations', 'best_match'],
      dtype='object')

In [37]:
df_best_match = df_best_match.drop(['titulo_tokens', 'sintesis_tokens',
       'texto_tokens', 'titulo_word_inds', 'titulo_case_inds',
       'sintesis_word_inds', 'sintesis_case_inds', 'texto_word_inds',
       'texto_case_inds', 'titulo_labels', 'sintesis_labels', 'texto_labels'
       ], axis=1)
df_best_match.head()

,Titulo,Sintesis,Texto,titulo_tags,sintesis_tags,texto_tags,titulo_locations,sintesis_locations,texto_locations,best_match
0,Exigen a TransCanada transparentar proyecto de...,Presidentes de las sociedades cooperativas pes...,Presidentes de las sociedades cooperativas pes...,"[[(Exigen, I-MISC), (a, I-MISC), (TransCanada,...","[[(Presidentes, I-MISC), (de, I-MISC), (las, I...","[[(Presidentes, I-MISC), (de, I-MISC), (las, I...",[],[],"[(Gasoducto, I-LOC), (Citlali, B-LOC), (Refiri...",Lorena
1,Violento fin de semana en dos estados de Mexic...,Los casos ocurrieron en Guanajuato y Chihuahua...,Los casos ocurrieron en Guanajuato y Chihuahua...,"[[(Violento, I-MISC), (fin, I-MISC), (de, I-MI...","[[(Los, I-MISC), (casos, I-MISC), (ocurrieron,...","[[(Los, I-MISC), (casos, I-MISC), (ocurrieron,...",[],"[(Mas, B-LOC)]","[(Mas, B-LOC), (Rocio, B-LOC), (Mayo, B-LOC), ...",Mas
2,Expulsan a 3 personas en Pahuatlan por trabaja...,Los empleados de la TransCanada regresaban de ...,Los empleados de la TransCanada regresaban de ...,"[[(Expulsan, I-MISC), (a, I-MISC), (3, I-MISC)...","[[(Los, I-MISC), (empleados, I-MISC), (de, I-M...","[[(Los, I-MISC), (empleados, I-MISC), (de, I-M...",[],[],"[(Presidente, B-LOC), (Rogelio, B-LOC), (Turis...",Presidente
4,Identifican a joven localizada sin vida en Gua...,La joven madre fue identificada como Rosario J...,La joven madre fue identificada como Rosario J...,"[[(Identifican, I-MISC), (a, I-MISC), (joven, ...","[[(La, I-MISC), (joven, I-MISC), (madre, I-MIS...","[[(La, I-MISC), (joven, I-MISC), (madre, I-MIS...",[],"[(Rosario, B-LOC)]","[(Rosario, B-LOC), (Chihuahua.-, B-LOC), (Rosa...",Rosario
5,Asaltan empresa de aluminio y roban vehiculo a...,No se registraron detenidos durante los hechos,"CELAYA, Gto.-Tres delincuentes armados asaltar...","[[(Asaltan, I-MISC), (empresa, I-MISC), (de, I...","[[(No, I-MISC), (se, I-MISC), (registraron, I-...","[[(CELAYA, B-LOC), (,, I-MISC), (Gto.-Tres, I-...",[],[],"[(CELAYA, B-LOC), (Pascual, B-LOC), (Reforma, ...",Ocampo


In [38]:
# Save predictions
df_best_match.to_csv(PREDICTIONS_PATH, sep=';')